## Submission instructions

All code that you write should be in this notebook. Please include your names and student numbers. You have to submit two files:

* This notebook with your code added. Make sure to add enough documentation.
* A short report, around 2 pages including any figures and/or tables (it is likely that you won't need the full 2 pages). Use [this template](https://www.overleaf.com/read/mvskntycrckw).

For questions, make use of the "Lab" session (see schedule).
Questions can also be posted to the MS teams channel called "Lab".

**Note:** You are free to make use of Python libraries (e.g., numpy, sklearn, etc.) except any *fairness* libraries.

#### Name and student numbers
...


## Dataset

In this assignment we are going to use the **COMPAS** dataset.

If you haven't done so already, take a look at this article: https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing.
For background on the dataset, see https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm.

**Reading in the COMPAS dataset**

The dataset can be downloaded here: https://github.com/propublica/compas-analysis/blob/master/compas-scores-two-years.csv

For this assignment, we focus on the protected attribute *race*.

The label (the variable we want to be able to predict) represents recidivism, which is defined as a new arrest within 2 years.

In [ ]:
!wget -c https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv

--2024-04-25 07:21:03--  https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2546489 (2.4M) [text/plain]
Saving to: ‘compas-scores-two-years.csv’

compas-scores-two-y 100%[===================>]   2.43M  --.-KB/s    in 0.07s   

2024-04-25 07:21:03 (35.4 MB/s) - ‘compas-scores-two-years.csv’ saved [2546489/2546489]



In [38]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import random

def set_seed(seed):
    #set all random seeds
    np.random.seed(seed)
    random.seed(seed)
    return None

SEED = 42
set_seed(SEED)

compas_data = pd.read_csv('compas-scores-two-years.csv')

We apply several data preprocessing steps, including only retaining Caucasians and African Americans.

In [39]:
compas_data = compas_data[(compas_data.days_b_screening_arrest <= 30)
            & (compas_data.days_b_screening_arrest >= -30)
            & (compas_data.is_recid != -1)
            & (compas_data.c_charge_degree != 'O')
            & (compas_data.score_text != 'N/A')
            & ((compas_data.race == 'Caucasian') | (compas_data.race == 'African-American'))]

Take a look at the data:

In [27]:
compas_data.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,...,1,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0
10,14,benjamin franc,benjamin,franc,2013-11-26,Male,1988-06-01,27,25 - 45,Caucasian,...,4,Low,2013-11-26,2013-11-25,2013-11-26,0,0,857,0,0


Now take a look at the distribution of the protected attribute `race` and the distribution of our outcome variable `two_year_recid`.

**Note:** in the context of fair machine learning, the favorable label here is no recidivism, i.e., ```two_year_recid = 0```

In [28]:
print('Number of instances per race category:')
print(compas_data[['race', 'two_year_recid']].value_counts())

Number of instances per race category:
race              two_year_recid
African-American  1                 1661
                  0                 1514
Caucasian         0                 1281
                  1                  822
Name: count, dtype: int64


## Data analysis

### **1. Exploration**

Perform an exploratory analysis of your data and describe the results in your report.
Include at least the following:

1. The size of your data;
2. The size of the protected attribute classes;
3. The base rates (the probability of a favorable outcome for the two protected attribute classes);
4. The base rates for the combination of both race and sex categories. What do you see?

### **2. Performance measures**

You will have to measure the performance and fairness of different classifiers in question 5. The performance will be calculated with the precision, recall, F1 and accuracy.
Additionally, you will have to calculate the statistical/demographic parity, the true positive rate (recall) and false positive rate per race group.

Make sure that you are able to calculate these metrics in the cell below.

In [87]:
# Your code for the performance measures
def metrics(df, groups='race', disparity_threshold=0.8):
    '''
    df: pandas DataFrame, must contain columns two_year_recid, prediction and column specified in groups (default "race")
    returns: dictionary with performance metrics
    '''
    performance = {}
    performance['accuracy'] = accuracy_score(df['two_year_recid'], df['prediction'])
    performance['precision'], performance['recall'], performance['f1'], _ = precision_recall_fscore_support(df['two_year_recid'], df['prediction'], average='binary')
    aa_group = df[df[groups] == 'African-American']
    cc_group = df[df[groups] == 'Caucasian']
    P_pos_aa = aa_group[aa_group['prediction'] == 0].shape[0] / (aa_group.shape[0] + 1e-6)
    P_pos_cc = cc_group[cc_group['prediction'] == 0].shape[0] / (cc_group.shape[0] + 1e-6)
    performance['disparate_impact'] = P_pos_aa/P_pos_cc
    performance['demographic_parity'] = P_pos_aa/P_pos_cc < disparity_threshold

    TP_rate_aa = aa_group[(aa_group['prediction'] == 0) & (aa_group['two_year_recid'] == 0)].shape[0] / (aa_group[aa_group['two_year_recid'] == 0].shape[0] + 1e-6)
    TP_rate_cc = cc_group[(cc_group['prediction'] == 0) & (cc_group['two_year_recid'] == 0)].shape[0] / (cc_group[cc_group['two_year_recid'] == 0].shape[0] + 1e-6)

    performance['African-American_true_positive_rate'] = TP_rate_aa
    performance['Caucasian_true_positive_rate'] = TP_rate_cc
    performance['equal_opportunity'] = TP_rate_aa/TP_rate_cc < disparity_threshold

    FP_rate_aa = aa_group[(aa_group['prediction'] == 0) & (aa_group['two_year_recid'] == 1)].shape[0] / (aa_group[aa_group['two_year_recid'] == 0].shape[0] + 1e-6)
    FP_rate_cc = cc_group[(cc_group['prediction'] == 0) & (cc_group['two_year_recid'] == 1)].shape[0] / (cc_group[cc_group['two_year_recid'] == 0].shape[0] + 1e-6)

    performance['African-American_false_positive_rate'] = FP_rate_aa
    performance['Caucasian_false_positive_rate'] = FP_rate_cc
    performance['predictive_parity'] = FP_rate_aa/FP_rate_cc < disparity_threshold

    # round to 4 decimal places
    for key in performance.keys():
        performance[key] = round(performance[key], 4)

    return performance  

In [41]:
# example usage
compas_data['prediction'] = np.random.randint(0, 2, compas_data.shape[0])

metrics(compas_data)

{'accuracy': 0.5064,
 'precision': 0.4768,
 'recall': 0.5046,
 'f1': 0.4903,
 'disparate_impact': 0.9967,
 'demographic_parity': 0,
 'African-American_true_positive_rate': 0.5099,
 'Caucasian_true_positive_rate': 0.5059,
 'equal_opportunity': 0,
 'African-American_false_positive_rate': 0.5416,
 'Caucasian_false_positive_rate': 0.3201,
 'predictive_parity': 0}

### **3. Prepare the data**
For the classifiers in question 5, the input of the model can only contain numerical values, it is therefore important to convert the strings in the columns (features) of interest of the `compas_data` to floats or integers.

The columns of interest are features that you think will be informative or interesting in predicting the outcome variable.

Use the cell below to explore which of the compas variables you need to convert to be able to use them for the classifiers. Also make sure that you understand the relationship between categorical and numerical values.

Then generate a new dataframe with your selected features in the right encoding (also make sure to include `two_year_recid`). You can implement this yourself, or use the `LabelEncoder` from `sklearn`.

**Note:** you do not need to convert all columns/features, only the ones you are interested in. Please include in your **report** a short motivation (one-two sentence) per feature why you think this is informative or interesting to take into account.

In [52]:
# Your code to prepare the data
# print data type of each column
print('initial dataset dat atypes', compas_data.dtypes)

obj_features = compas_data.select_dtypes(include='object').columns
print('list of non numerical features:', obj_features)

date_time_features = ['compas_screening_date', 'dob', 'c_jail_in', 'c_jail_out',
       'c_offense_date', 'c_arrest_date', 'r_offense_date',
       'r_jail_in', 'r_jail_out', 'screening_date', 'in_custody', 'out_custody']

non_train_features = ['id', 'c_case_number', 'r_case_number', 'vr_case_number', 'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc']

categorical_features = list(set(obj_features) - set(date_time_features) - set(non_train_features))
categorical_features
print('list of categorical features:', categorical_features)

initial dataset dat atypes id                           int64
name                        object
first                       object
last                        object
compas_screening_date       object
sex                         object
dob                         object
age                          int64
age_cat                     object
race                        object
juv_fel_count                int64
decile_score                 int64
juv_misd_count               int64
juv_other_count              int64
priors_count                 int64
days_b_screening_arrest    float64
c_jail_in                   object
c_jail_out                  object
c_case_number               object
c_offense_date              object
c_arrest_date               object
c_days_from_compas         float64
c_charge_degree             object
c_charge_desc               object
is_recid                     int64
r_case_number               object
r_charge_degree             object
r_days_from_arrest         f

In [53]:
compas_data[date_time_features].isna().sum()

compas_screening_date       0
dob                         0
c_jail_in                   0
c_jail_out                  0
c_offense_date            689
c_arrest_date            4589
r_offense_date           2631
r_jail_in                3497
r_jail_out               3497
screening_date              0
in_custody                  0
out_custody                 0
dtype: int64

In [72]:
non_ohe_ftrs = list(set(X_train.columns) - set(processor.encoder.get_feature_names_out()))
X_train[non_ohe_ftrs].isna().sum()

c_jail_in                     0
is_violent_recid              0
start                         0
r_jail_in                     0
r_jail_out_missing            0
v_decile_score                0
r_days_from_arrest         2809
c_jail_out                    0
c_arrest_date_missing         0
out_custody                   0
juv_fel_count                 0
in_custody                    0
decile_score                  0
age                           0
r_jail_in_missing             0
days_b_screening_arrest       0
c_offense_date_missing        0
decile_score.1                0
priors_count                  0
screening_date                0
c_offense_date                0
is_recid                      0
c_days_from_compas            0
violent_recid              4222
r_jail_out                    0
priors_count.1                0
r_offense_date_missing        0
c_arrest_date                 0
juv_other_count               0
event                         0
juv_misd_count                0
r_offens

In [73]:
def dt_to_seconds(series):
    series = pd.to_datetime(series)
    return (series - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

class FeaturePreprocessor:
    def __init__(self, categorical_features, date_time_features, non_train_features, min_freq=0.01):
        self.categorical_features = categorical_features
        self.date_time_features = date_time_features
        self.non_train_features = non_train_features
        self.scaler = StandardScaler()
        self.encoder = OneHotEncoder(sparse_output=False, min_frequency=min_freq)

    def fit_transform(self, X, full_cetagorical_df):
        X = X.copy()
        X = X.drop(columns=self.non_train_features)
        self.numerical_features = list(set(X.columns) - set(self.categorical_features))
        for ftr in self.date_time_features:
            X[ftr] = dt_to_seconds(X[ftr])
            if X[ftr].isna().sum() > 0:
                new_ftr = ftr + '_missing'
                X[new_ftr] = X[ftr].isna().astype(int)
                X[ftr] = X[ftr].fillna(-1)
        
        X[self.numerical_features] = self.scaler.fit_transform(X[self.numerical_features])
        self.encoder.fit(full_cetagorical_df)
        ohe_feature_names = self.encoder.get_feature_names_out()
        X[ohe_feature_names] = self.encoder.transform(X[self.categorical_features])
        X = X.drop(columns=self.categorical_features)
        for col in ['r_days_from_arrest', 'violent_recid']:
            if col in X.columns:
                X[col] = X[col].fillna(0)
        return X


    def transform(self, X):
        X = X.copy()
        X = X.drop(columns=self.non_train_features)
        for ftr in self.date_time_features:
            X[ftr] = dt_to_seconds(X[ftr])
            if X[ftr].isna().sum() > 0:
                new_ftr = ftr + '_missing'
                X[new_ftr] = X[ftr].isna().astype(int)
                X[ftr] = X[ftr].fillna(-1)
        
        X[self.numerical_features] = self.scaler.transform(X[self.numerical_features])
        ohe_feature_names = self.encoder.get_feature_names_out()
        X[ohe_feature_names] = self.encoder.transform(X[self.categorical_features])
        X = X.drop(columns=self.categorical_features)
        for col in ['r_days_from_arrest', 'violent_recid']:
            if col in X.columns:
                X[col] = X[col].fillna(0)
        return X

### **4. Train and test split**

Divide the dataset into a train (80%) and test split (20%), either by implementing it yourself, or by using an existing library.

**Note:** Usually when carrying out machine learning experiments,
we also need a dev set for developing and selecting our models (incl. tuning of hyper-parameters).
However, in this assignment, the goal is not to optimize
the performance of models so we'll only use a train and test split.




In [91]:
X, y = compas_data.drop(columns=['two_year_recid']), compas_data['two_year_recid']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [92]:
processor = FeaturePreprocessor(categorical_features, date_time_features, non_train_features)
X_train = processor.fit_transform(X_train, X[categorical_features])
X_test = processor.transform(X_test)

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/var/folders/wl/tkx29dw96sbgly88cthpjgzc0000gn/T/ipykernel_35310/273650697.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[ohe_feature_names] = self.encoder.transform(X[self.categorical_fe

### **5. Classifiers**

Now, train and test different classifiers and report the following statistics:

* Overall precision, recall, F1 and accuracy;
* The statistical parity difference for the protected attribute (difference between the probability of a favorable label);
* The true positive rates and the false positive rates of the two protected attribute groups.

For training the classifier we recommend using scikit-learn (https://scikit-learn.org/stable/).

#### **5.1 Regular classification**
Train a logistic regression classifier with the race feature and all other features that you are interested in.

In [93]:
logreg = LogisticRegression(random_state=SEED)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
test_res_df = X_test.copy()
test_res_df['race'] = 'Afracan-American'
test_res_df.loc[test_res_df['race_Caucasian'] == 1, 'race'] = 'Caucasian'
test_res_df['two_year_recid'] = y_test

test_res_df['prediction'] = y_pred
metrics(test_res_df)

{'accuracy': 0.982,
 'precision': 0.976,
 'recall': 0.9859,
 'f1': 0.9809,
 'disparate_impact': 0.0,
 'demographic_parity': 1,
 'African-American_true_positive_rate': 0.0,
 'Caucasian_true_positive_rate': 0.9755,
 'equal_opportunity': 1,
 'African-American_false_positive_rate': 0.0,
 'Caucasian_false_positive_rate': 0.0204,
 'predictive_parity': 1}

#### **5.2 Without the protected attribute**
Train a logistic regression classifier without the race feature, but with all other features you used in 5.1.


In [94]:
X_train_protected = X_train.drop(['race_African-American', 'race_Caucasian'], axis=1)
X_test_protected = X_test.drop(['race_African-American', 'race_Caucasian'], axis=1)

logreg = LogisticRegression(random_state=SEED)
logreg.fit(X_train_protected, y_train)
y_pred = logreg.predict(X_test_protected)

test_res_df['prediction'] = y_pred
metrics(test_res_df)

{'accuracy': 0.9811,
 'precision': 0.976,
 'recall': 0.9839,
 'f1': 0.9799,
 'disparate_impact': 0.0,
 'demographic_parity': 1,
 'African-American_true_positive_rate': 0.0,
 'Caucasian_true_positive_rate': 0.9755,
 'equal_opportunity': 1,
 'African-American_false_positive_rate': 0.0,
 'Caucasian_false_positive_rate': 0.0204,
 'predictive_parity': 1}

#### **5.3 Pre-processing: Reweighing**
Train a classifier with weights (see lecture slides the weight calculation)
* Report the weights that are used for reweighing and a short interpretation/discussion.
* Hint: Think about when you should reweight the data: during initialization or during training (i.e. fit)? Read the documentation of the Logistic regression model in Scikit-learn carefully (if you use it).

In [95]:
data = X_train.copy()
data['outcome'] = y_train

# Calculate the necessary proportions
# Proportions of positive outcomes
P_AA_pos = (data['race_African-American'] == 1) & (data['outcome'] == 0)
P_C_pos = (data['race_Caucasian'] == 1) & (data['outcome'] == 0)

# Proportions of negative outcomes
P_AA_neg = (data['race_African-American'] == 1) & (data['outcome'] == 1)
P_C_neg = (data['race_Caucasian'] == 1) & (data['outcome'] == 1)

# Convert booleans to integers and calculate the proportion
p1_pos = P_AA_pos.sum() / (data['race_African-American'] == 1).sum()
p2_pos = P_C_pos.sum() / (data['race_Caucasian'] == 1).sum()
p1_neg = P_AA_neg.sum() / (data['race_African-American'] == 1).sum()
p2_neg = P_C_neg.sum() / (data['race_Caucasian'] == 1).sum()

# Desired balance for each outcome across all groups
total_pos = data['outcome'].value_counts(normalize=True)[0]  # total proportion of positive outcomes
total_neg = data['outcome'].value_counts(normalize=True)[1]  # total proportion of negative outcomes

# Calculate weights
data['weight'] = np.where(data['race_African-American'] == 1,
                          np.where(data['outcome'] == 0, (total_pos / p1_pos), (total_neg / p1_neg)),
                          np.where(data['outcome'] == 0, (total_pos / p2_pos), (total_neg / p2_neg)))

In [96]:
logreg = LogisticRegression()
logreg.fit(X_train_protected, y_train, sample_weight=data['weight'])

LogisticRegression()

In [97]:
y_pred = logreg.predict(X_test_protected)

test_res_df['prediction'] = y_pred
metrics(test_res_df)

{'accuracy': 0.9801,
 'precision': 0.976,
 'recall': 0.9819,
 'f1': 0.9789,
 'disparate_impact': 0.0,
 'demographic_parity': 1,
 'African-American_true_positive_rate': 0.0,
 'Caucasian_true_positive_rate': 0.9755,
 'equal_opportunity': 1,
 'African-American_false_positive_rate': 0.0,
 'Caucasian_false_positive_rate': 0.0204,
 'predictive_parity': 1}

#### **5.4 Post-processing: Equalized odds**
Use the predictions by the first classifier for this post processing part (see lecture slides for more information about post processing for equalized odds).

Normally, the best parameters `p_00, p_01, p_10, p_11` are found with a linear program that minimizes loss between predictions of a derived predictor and the actual labels. In this assignment we will not ask you to do this. Instead, we would like you to follow the next steps to find parameters, post-process the data and check the performance of this classifier with post-processing:

1. Generate a 1000 different samples of these 4 parameters randomly;
2. Write a function (or more) that applies these 4 parameters to postprocess the predictions.
3. For each generated set of 4 parameters:
  - Change the predicted labels with the function(s) from step 2;
  - Calculate the TPR ratio and FPR ratio over these 'new' predictions;
4. Which set of parameters satisfies the equalized odds fairness measure the best?
5. Check the overall performance (precision, recall, accuracy, etc.) of the new predictions after post-processing and add an interpretation of the results and the parameters in your **report**.

In [ ]:
# Your code for step 1


In [ ]:
# Your code for step 2


In [ ]:
# Your code for step 3


In [9]:
# Your code for step 4 and 5


#### **Results**
For each classifier write in the **report**:
- Whether this classifier satisfies statistical parity and whether this compares to the original dataset;
- Does the classifier satisfy the equal opportunity criterion?
- How do the different classifiers compare against each other?

### **6. Intersectional fairness**
In the questions above `race` was the only protected attribute. However, multiple protected attributes sometimes interact, leading to different fairness outcomes for different combinations of these protected attributes.

Please explore the intersectional fairness for protected attributes `race` and `sex` for the first two classifiers from question 5: instead of only reporting the statistical parity, TPR and FPR for the protected attribute race, make a combination of the `race` and `sex` column, with four categories of this combined protected attribute, and report the maximum difference between the subgroups for statistical parity, TPR and FPR. 
For example, suppose we have four groups with TPRs 0.1, 0.2, 0.8, then the maximum difference is 0.7.
- For classifier 1: evaluate the predictions for this combined protected attribute;
- For classifier 2: remove both `race` and `sex` from the data and check the results again.

Write a short interpretation of these results in your **report**.

In [ ]:
# Your code for intersectional fairness


## Discussion
Include a short ethical discussion (1 or 2 paragraphs) in your **report** reflecting on these two aspects:

1) The use of a ML system to try to predict recidivism;

2) The public release of a dataset like this.